In [1]:
import numpy as np
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, Reshape, Input
from keras.layers import LSTM
from keras.initializers import RandomNormal
from keras.callbacks import ModelCheckpoint, EarlyStopping
import json
import csv
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
file_path = "./singlerack_all.csv"
server_list = []
conditioner_outlet_temp = []
conditioner_inlet_temp = []


timestep = 36
predict_horizon = 12

class Server(object):
    def __init__(self, i):
        self.id = i
        self.inlet_temp = []
        self.outlet_temp = []
        self.cpu = []
        self.memory = []


def strided_app(a, L, S):  # Window len = L, Stride len/stepsize = S
    nrows = ((a.size - L) // S) + 1
    n = a.strides[0]
    return np.lib.stride_tricks.as_strided(a, shape=(nrows, L), strides=(S * n, n), writeable=False)


def process_data():
    for i in range(15):
        server_list.append(Server(i))
    with open(file_path, "r", encoding='utf-8') as datacsv:
        csvr = csv.reader(datacsv)
        for row in csvr:
            i = 1
            for server in server_list:
                server.outlet_temp.append(float(row[i]))
                i = i + 1
            for server in server_list:
                server.inlet_temp.append(float(row[46 - i]))
                i = i + 1
            conditioner_outlet_temp.append(float(row[i]))
            i = i + 1
            conditioner_inlet_temp.append(float(row[i]))
            i = i + 6
            for server in server_list:
                # if(server.id<10):
                #     server.cpu.append(float(row[i])/10)
                # else:
                #     server.cpu.append(float(row[i]))
                server.cpu.append(float(row[i]) / 100)
                i = i + 6

process_data()



In [2]:
%matplotlib qt5
from keras.models import load_model
import matplotlib.pyplot as plt
i=0
for server in server_list:
    i+=1
    plt.plot(np.arange(len(server.inlet_temp)),server.inlet_temp,label=str(i))
plt.plot(np.arange(len(conditioner_inlet_temp)),conditioner_inlet_temp,label="c_in")
plt.plot(np.arange(len(conditioner_outlet_temp)),conditioner_outlet_temp,label="c_out")
plt.legend()
plt.show()



In [5]:
temp_train_x = []
temp_train_y = []
server=server_list[13]
temp = np.array(server.inlet_temp)
data = strided_app(temp, timestep + predict_horizon + 1, 1)
x = data[:, :-1 - predict_horizon]
y = data[:, -1]
t_conditioner_outlet_temp = np.array(conditioner_outlet_temp)[48:]
t_conditioner_outlet_temp = t_conditioner_outlet_temp.reshape(-1, 1)
x = np.concatenate((x, t_conditioner_outlet_temp), axis=1)
if isinstance(temp_train_x, list):
    temp_train_x = x
    temp_train_y = y
else:
    temp_train_x = np.concatenate((temp_train_x, x), axis=0)
    temp_train_y = np.concatenate((temp_train_y, y), axis=0)
cpu_x = []

cpu = np.array(server.cpu)
data = strided_app(cpu, timestep + predict_horizon + 1, 1)
x = data[:, :-1 - predict_horizon]
if isinstance(cpu_x, list):
    cpu_x = x
else:
    cpu_x = np.concatenate((cpu_x, x), axis=0)

cpu_pre_model = load_model("./model/predict_cpu_ts36_ph12_0.17.hdf5")
cpu_pre_result = cpu_pre_model.predict(cpu_x)
mixed_train_x = np.concatenate((temp_train_x, cpu_pre_result), axis=1)
temp_pre_model=load_model("./model/predict_temp_usingcpu+condition_ts36_ph12_0.15.hdf5")
temp_pre_res=temp_pre_model.predict(mixed_train_x)


In [4]:
temp_cpu_combined_x=np.concatenate((temp_train_x,cpu_x),axis=1)
temp_cpu_combined_model=load_model("./model/predict_temp_combined_cpu+temp_ts36_ph12_0.22.hdf5")
temp_cpu_combined_y=temp_cpu_combined_model.predict(temp_cpu_combined_x)


In [17]:
%matplotlib qt5
plt.plot(np.arange(len(server.inlet_temp[2730:3230])),server.inlet_temp[2730:3230],label='origin')
plt.plot(np.arange(len(temp_pre_res[2700:3200])),temp_pre_res[2700:3200],label='double lstm')
plt.plot(np.arange(len(temp_cpu_combined_y[2700:3200])),temp_cpu_combined_y[2700:3200],label='single lstm')
plt.legend()
plt.show()


In [6]:
temp_train_x = []
temp_train_y = []

temp = np.array(server.inlet_temp)
data = strided_app(temp, timestep + predict_horizon + 1, 1)
x = data[:, :-1 - predict_horizon]
y = data[:, -1]
t_conditioner_outlet_temp = np.array(conditioner_outlet_temp)[48:]
t_conditioner_outlet_temp = t_conditioner_outlet_temp.reshape(-1, 1)
x = np.concatenate((x, t_conditioner_outlet_temp), axis=1)
if isinstance(temp_train_x, list):
    temp_train_x = x
    temp_train_y = y
else:
    temp_train_x = np.concatenate((temp_train_x, x), axis=0)
    temp_train_y = np.concatenate((temp_train_y, y), axis=0)
cpu_x = []

cpu = np.array(server.cpu)
data = strided_app(cpu, timestep + predict_horizon + 1, 1)
x = data[:, :-1 - predict_horizon]
if isinstance(cpu_x, list):
    cpu_x = x
else:
    cpu_x = np.concatenate((cpu_x, x), axis=0)

# cpu_pre_model = train_cpu_model(timestep, predict_horizon)
from keras.models import load_model
cpu_pre_model = load_model("./model/predict_cpu_ts36_ph12_0.19.hdf5")
cpu_pre_result = cpu_pre_model.predict(cpu_x)
mixed_train_x = np.concatenate((temp_train_x, cpu_pre_result), axis=1)
temp_pre_model_cpu_conditon=load_model("./model/predict_temp_usingcpu+condition_ts36_ph12_0.16.hdf5")

temp_pre_model_cpu_y=temp_pre_model_cpu_conditon.predict(mixed_train_x)

In [12]:
%matplotlib qt5
plt.plot(np.arange(len(server.inlet_temp[2836:3436])),server.inlet_temp[2836:3436],label='origin')
plt.plot(np.arange(len(temp_pre_model_cpu_y[2800:3400])),temp_pre_model_cpu_y[2800:3400],label='double lstm')
plt.plot(np.arange(len(temp_cpu_combined_y[2800:3400])),temp_cpu_combined_y[2800:3400],label='single lstm')
plt.xlabel("time(s)")
plt.ylabel("temperature(°C)")
plt.legend()
plt.show()


In [19]:
"""
    @author: samuel ko
    @date:   2018/12/17
    @target: 研究模型的中间输出结果
    @ref: 作者：挥挥洒洒
          来源：CSDN
          原文：https://blog.csdn.net/u010420283/article/details/80303231
"""
from keras.models import load_model
from keras import backend as K
import numpy as np
h_tm_i, h_tm_o, h_tm_c, h_tm_f, c_tm = None, None, None, None, None


def hard_sigmoid(x):
    x = 0.2 * x + 0.5
    x[x < -2.5] = 0
    x[x > 2.5] = 1
    return x


def lstm_keras_verify(inputs):
    global h_tm_c, h_tm_f, h_tm_i, h_tm_o, c_tm
    # kernel初始化
    kernel_i = np.array([0.4309869408607483, 1.184934139251709, 1.1755656003952026, 0.29152509570121765, 0.9355264902114868])
    kernel_f = np.array([0.4721968472003937, 0.8939654231071472, 0.3940809667110443, 0.32647714018821716, 0.3925175964832306])
    kernel_c = np.array([0.43232300877571106, 0.9761391282081604, 0.4974423944950104, -0.5713692307472229, 0.6272905468940735])
    kernel_o = np.array([0.4851478338241577, 0.4159347116947174, 0.8334378600120544, 0.6494604349136353, 1.4963207244873047])
    print(kernel_i.shape,kernel_f.shape,kernel_c.shape,kernel_o.shape)
    recurrent_kernel_i = np.array([[-0.15266947448253632, -0.4967867434024811, -0.2602699398994446, -0.3376578092575073, 0.18315182626247406],
                          [0.40668627619743347, 0.11702277511358261, 0.2870166599750519, -0.09417486935853958, 1.2248116731643677],
                          [0.13948452472686768, -0.2935984432697296, -0.18430666625499725, 0.04545489326119423, 0.8304147720336914],
                          [-0.9957871437072754, -1.2020113468170166, -1.1591960191726685, -0.2052622139453888, -1.3381662368774414],
                          [1.1894947290420532, 0.675262451171875, 0.6069576144218445, 0.5705539584159851, 0.9218697547912598]])

    recurrent_kernel_f = np.array([[-0.548134982585907, -0.12552201747894287, -0.41158366203308105, 0.09746172279119492, 0.19226618111133575],
                          [0.10524879395961761, 0.032132066786289215, 0.0605274997651577, 0.07235733419656754, 0.7413577437400818],
                          [-0.17540045082569122, -0.40539026260375977, -0.18782351911067963, 0.20610281825065613, 0.8710744380950928],
                          [-0.7760279178619385, -0.9006417393684387, -0.7003670334815979, -0.22393617033958435, -0.5202550888061523],
                          [0.7772086262702942, 0.7663999199867249, 0.5117960572242737, 0.13461880385875702, 0.7836397290229797]])

    recurrent_kernel_c = np.array([[1.580788493156433, 1.0911318063735962, 0.6749269366264343, 0.30827417969703674, 0.7559695839881897],
                          [0.7300652265548706, 0.9139286875724792, 1.1172183752059937, 0.043491244316101074, 0.8009109497070312],
                          [1.49398934841156, 0.5944592356681824, 0.8874677419662476, -0.1583320051431656, 1.3592860698699951],
                          [0.032015360891819, -0.5035645365715027, -0.3792402148246765, 0.42566269636154175, -0.6349631547927856],
                          [0.12018230557441711, 0.33967509865760803, 0.5114297270774841, -0.062018051743507385, 0.5401539206504822]])

    recurrent_kernel_o = np.array([[-0.41055813431739807, -0.017661772668361664, 0.06882145255804062, 0.09856614470481873, 0.44098445773124695],
                          [0.5692929625511169, 0.5409368872642517, 0.3319447338581085, 0.4997922480106354, 0.9462743401527405],
                          [0.1794481724500656, 0.10621143877506256, -0.0016202644910663366, -0.010369917377829552, 0.4268817901611328],
                          [-1.026210904121399, -0.6898611783981323, -0.9652346968650818, -0.07141508907079697, -0.6710768938064575],
                          [0.5829002261161804, 0.6890853047370911, 0.5738061666488647, -0.16630153357982635, 1.2376824617385864]])
    print(recurrent_kernel_i.shape,recurrent_kernel_f.shape,recurrent_kernel_c.shape,recurrent_kernel_o.shape)
    bias_i = np.array([1.1197513341903687, 1.0861579179763794, 1.0329890251159668, 0.3536357581615448, 0.9598652124404907])
    bias_f = np.array([2.020589828491211, 1.940927267074585, 1.9546188116073608, 1.1743367910385132, 1.7189750671386719])
    bias_c = np.array([-0.41391095519065857, -0.21292796730995178, -0.30117690563201904, -0.24005982279777527, 0.053657304495573044])
    bias_o = np.array([1.222458004951477, 1.1024200916290283, 1.0836670398712158, 0.3483290672302246, 0.9281882643699646])
    print(bias_i.shape,bias_f.shape,bias_c.shape,bias_o.shape)
    # step 1 计算W * x
    x_i = inputs * kernel_i
    x_f = inputs * kernel_f
    x_c = inputs * kernel_c
    x_o = inputs * kernel_o

    # step 2 加上bias
    x_i += bias_i
    x_f += bias_f
    x_c += bias_c
    x_o += bias_o

    # step 3 计算
    if not isinstance(h_tm_i, np.ndarray):
        h_tm_i = np.zeros((1, 5))
        h_tm_o = np.zeros((1, 5))
        h_tm_f = np.zeros((1, 5))
        h_tm_c = np.zeros((1, 5))
        c_tm = np.zeros((1, 5))
    i = hard_sigmoid(x_i + np.dot(h_tm_i, recurrent_kernel_i))
    f = hard_sigmoid(x_f + np.dot(h_tm_f, recurrent_kernel_f))
    c = f * c_tm + i * np.tanh(x_c + np.dot(h_tm_c, recurrent_kernel_c))
    o = hard_sigmoid(x_o + np.dot(h_tm_o, recurrent_kernel_o))

    h = o * np.tanh(c)

    h_tm_c = h_tm_f = h_tm_o = h_tm_i = h
    c_tm = c

    print("当前的hidden state", h)
    print("当前的cell state", c)
    return h, c
inputs = np.array([[0], [0.03846154], [0.07692308]])
inputs = np.expand_dims(inputs, 0)
lstm_keras_verify(inputs)


(5,) (5,) (5,) (5,)
(5, 5) (5, 5) (5, 5) (5, 5)
(5,) (5,) (5,) (5,)
当前的hidden state [[[-0.20567793 -0.10758754 -0.14600677 -0.07612558  0.02542126]
  [-0.20047084 -0.09077541 -0.14040491 -0.08375611  0.03780037]
  [-0.19501361 -0.07297879 -0.13433439 -0.09145965  0.05075057]]]
当前的cell state [[[-0.2836353  -0.15045176 -0.20660162 -0.13443607  0.03709382]
  [-0.27463141 -0.12609938 -0.19665536 -0.14679395  0.05427468]
  [-0.26540282 -0.10073903 -0.18625618 -0.15911148  0.07173786]]]


(array([[[-0.20567793, -0.10758754, -0.14600677, -0.07612558,
           0.02542126],
         [-0.20047084, -0.09077541, -0.14040491, -0.08375611,
           0.03780037],
         [-0.19501361, -0.07297879, -0.13433439, -0.09145965,
           0.05075057]]]),
 array([[[-0.2836353 , -0.15045176, -0.20660162, -0.13443607,
           0.03709382],
         [-0.27463141, -0.12609938, -0.19665536, -0.14679395,
           0.05427468],
         [-0.26540282, -0.10073903, -0.18625618, -0.15911148,
           0.07173786]]]))

In [21]:
h_tm_i = np.zeros((1, 5))
h_tm_o = np.zeros((1, 5))
h_tm_f = np.zeros((1, 5))
h_tm_c = np.zeros((1, 5))
c_tm = np.zeros((1, 5))
kernel_i = np.array([0.4309869408607483, 1.184934139251709, 1.1755656003952026, 0.29152509570121765, 0.9355264902114868])
kernel_f = np.array([0.4721968472003937, 0.8939654231071472, 0.3940809667110443, 0.32647714018821716, 0.3925175964832306])
kernel_c = np.array([0.43232300877571106, 0.9761391282081604, 0.4974423944950104, -0.5713692307472229, 0.6272905468940735])
kernel_o = np.array([0.4851478338241577, 0.4159347116947174, 0.8334378600120544, 0.6494604349136353, 1.4963207244873047])
print(kernel_i.shape,kernel_f.shape,kernel_c.shape,kernel_o.shape)
recurrent_kernel_i = np.array([[-0.15266947448253632, -0.4967867434024811, -0.2602699398994446, -0.3376578092575073, 0.18315182626247406],
                      [0.40668627619743347, 0.11702277511358261, 0.2870166599750519, -0.09417486935853958, 1.2248116731643677],
                      [0.13948452472686768, -0.2935984432697296, -0.18430666625499725, 0.04545489326119423, 0.8304147720336914],
                      [-0.9957871437072754, -1.2020113468170166, -1.1591960191726685, -0.2052622139453888, -1.3381662368774414],
                      [1.1894947290420532, 0.675262451171875, 0.6069576144218445, 0.5705539584159851, 0.9218697547912598]])

recurrent_kernel_f = np.array([[-0.548134982585907, -0.12552201747894287, -0.41158366203308105, 0.09746172279119492, 0.19226618111133575],
                      [0.10524879395961761, 0.032132066786289215, 0.0605274997651577, 0.07235733419656754, 0.7413577437400818],
                      [-0.17540045082569122, -0.40539026260375977, -0.18782351911067963, 0.20610281825065613, 0.8710744380950928],
                      [-0.7760279178619385, -0.9006417393684387, -0.7003670334815979, -0.22393617033958435, -0.5202550888061523],
                      [0.7772086262702942, 0.7663999199867249, 0.5117960572242737, 0.13461880385875702, 0.7836397290229797]])

recurrent_kernel_c = np.array([[1.580788493156433, 1.0911318063735962, 0.6749269366264343, 0.30827417969703674, 0.7559695839881897],
                      [0.7300652265548706, 0.9139286875724792, 1.1172183752059937, 0.043491244316101074, 0.8009109497070312],
                      [1.49398934841156, 0.5944592356681824, 0.8874677419662476, -0.1583320051431656, 1.3592860698699951],
                      [0.032015360891819, -0.5035645365715027, -0.3792402148246765, 0.42566269636154175, -0.6349631547927856],
                      [0.12018230557441711, 0.33967509865760803, 0.5114297270774841, -0.062018051743507385, 0.5401539206504822]])

recurrent_kernel_o = np.array([[-0.41055813431739807, -0.017661772668361664, 0.06882145255804062, 0.09856614470481873, 0.44098445773124695],
                      [0.5692929625511169, 0.5409368872642517, 0.3319447338581085, 0.4997922480106354, 0.9462743401527405],
                      [0.1794481724500656, 0.10621143877506256, -0.0016202644910663366, -0.010369917377829552, 0.4268817901611328],
                      [-1.026210904121399, -0.6898611783981323, -0.9652346968650818, -0.07141508907079697, -0.6710768938064575],
                      [0.5829002261161804, 0.6890853047370911, 0.5738061666488647, -0.16630153357982635, 1.2376824617385864]])
print(recurrent_kernel_i.shape,recurrent_kernel_f.shape,recurrent_kernel_c.shape,recurrent_kernel_o.shape)
bias_i = np.array([1.1197513341903687, 1.0861579179763794, 1.0329890251159668, 0.3536357581615448, 0.9598652124404907])
bias_f = np.array([2.020589828491211, 1.940927267074585, 1.9546188116073608, 1.1743367910385132, 1.7189750671386719])
bias_c = np.array([-0.41391095519065857, -0.21292796730995178, -0.30117690563201904, -0.24005982279777527, 0.053657304495573044])
bias_o = np.array([1.222458004951477, 1.1024200916290283, 1.0836670398712158, 0.3483290672302246, 0.9281882643699646])
print(bias_i.shape,bias_f.shape,bias_c.shape,bias_o.shape)
inputs = np.array([[0], [0.03846154], [0.07692308]])
inputs = np.expand_dims(inputs, 0)
x_i=inputs*kernel_i
print(x_i.shape)
h_tm_i=np.dot(h_tm_i, recurrent_kernel_i)
print(h_tm_i.shape)

(5,) (5,) (5,) (5,)
(5, 5) (5, 5) (5, 5) (5, 5)
(5,) (5,) (5,) (5,)
(1, 3, 5)
(1, 5)
